In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install websockets
!pip install lxml

In [ ]:
import nest_asyncio
from ws_client import generate_websocket_url
import websockets
from websockets import client
import asyncio, websockets, json
import xml.etree.ElementTree as ET
from tools import run_request_from_file
from local_secrets import ENTS, HOST, URI, secrets
from ws_client import generate_websocket_url

In [ ]:
nest_asyncio.apply()

In [ ]:
url = generate_websocket_url(URI, 'GET', secrets, HOST)

In [ ]:
os.environ['BBG_UUID'] = '26656679'
os.environ['PX_NUM'] = '4571'
os.environ['USE_NEW_CMGR_XML'] = 'TRUE'
os.environ['USING_IBM_MQ'] = 'FALSE'

In [ ]:
import pandas as pd
import random
from base import Side, OrderType, SecurityIdType, SingleAllocation

def build_orders_from_df(crossed_df: pd.DataFrame, remaining_df: pd.DataFrame) -> list[dict]:
    """
    Build list_of_orders for BasketOrderXMLHanlder.

    - remaining_df: one order per row, account = portfolio_id
    - crossed_df:   two orders per row (BUY for buyer_portfolio, SELL for seller_portfolio)
    """

    orders: list[dict] = []

    # -------- helpers --------
    def norm_secid(raw: str | None) -> str | None:
        if not raw or not isinstance(raw, str):
            return None
        s = raw.strip()
        if not s:
            return None
        if not s.lower().endswith(" equity"):
            s = f"{s} Equity"
        return s

    def pick_side(raw, qty):
        # prefer text; fall back to qty sign
        if isinstance(raw, str):
            r = raw.strip().lower()
            if r in ("buy", "b", "1", "long"):
                return Side.BUY
            if r in ("sell", "s", "2", "short"):
                return Side.SELL
        try:
            return Side.SELL if float(qty) < 0 else Side.BUY
        except Exception:
            return Side.BUY

    def clean_qty(x, default=1000):
        try:
            return int(abs(float(x)))
        except Exception:
            return default

    # -------- remaining_df → 1 order per row --------
    if remaining_df is not None and not remaining_df.empty:
        # best-guess columns
        sec_cols   = ["security", "security_id", "symbol", "Ticker", "ticker"]
        qty_cols   = ["remaining_quantity", "quantity", "original_quantity"]
        side_cols  = ["trade_direction", "side", "direction"]
        acct_col   = "portfolio_id"

        def first_col(df, cands):
            for c in cands:
                if c in df.columns:
                    return c
            return None

        sec_col  = first_col(remaining_df, sec_cols)
        qty_col  = first_col(remaining_df, qty_cols)
        side_col = first_col(remaining_df, side_cols)

        if not sec_col or not qty_col or acct_col not in remaining_df.columns:
            raise ValueError("remaining_df is missing one of: security, quantity, portfolio_id")

        for _, row in remaining_df.iterrows():
            secid = norm_secid(row.get(sec_col))
            if not secid:
                continue

            qty  = clean_qty(row.get(qty_col))
            side = pick_side(row.get(side_col), row.get(qty_col))
            acct = str(row.get(acct_col)).strip()

            price = round(random.uniform(10, 250), 2)
            ccy   = "USD"

            orders.append(dict(
                security_id=secid,
                security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
                side=side,
                order_type=OrderType.LIMIT,
                limit_price=price,
                quantity=qty,
                settl_currency=ccy,
                allocation_instruction=[SingleAllocation(Account=acct, Quantity=qty)],
                crossed=False,  # no broker for remaining
            ))

    # -------- crossed_df → 2 orders per row (BUY + SELL) --------
    if crossed_df is not None and not crossed_df.empty:
        # best-guess columns
        sec_cols  = ["security", "security_id", "symbol", "Ticker", "ticker"]
        qty_cols  = ["quantity_crossed", "quantity", "original_quantity"]
        buyer_acct_col  = "buyer_portfolio"
        seller_acct_col = "seller_portfolio"

        def first_col(df, cands):
            for c in cands:
                if c in df.columns:
                    return c
            return None

        sec_col = first_col(crossed_df, sec_cols)
        qty_col = first_col(crossed_df, qty_cols)

        if not sec_col or not qty_col or buyer_acct_col not in crossed_df.columns or seller_acct_col not in crossed_df.columns:
            raise ValueError("crossed_df is missing one of: security, quantity, buyer_portfolio, seller_portfolio")

        for _, row in crossed_df.iterrows():
            secid = norm_secid(row.get(sec_col))
            if not secid:
                continue

            qty   = clean_qty(row.get(qty_col))
            buyer = str(row.get(buyer_acct_col)).strip()
            seller= str(row.get(seller_acct_col)).strip()

            price = round(random.uniform(10, 250), 2)
            ccy   = "USD"

            # BUY order for the buyer account
            orders.append(dict(
                security_id=secid,
                security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
                side=Side.BUY,
                order_type=OrderType.LIMIT,
                limit_price=price,
                quantity=qty,
                settl_currency=ccy,
                allocation_instruction=[SingleAllocation(Account=buyer, Quantity=qty)],
                crossed=True,  # add broker + parties
            ))

            # SELL order for the seller account
            orders.append(dict(
                security_id=secid,
                security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
                side=Side.SELL,
                order_type=OrderType.LIMIT,
                limit_price=price,
                quantity=qty,
                settl_currency=ccy,
                allocation_instruction=[SingleAllocation(Account=seller, Quantity=qty)],
                crossed=True,  # add broker + parties
            ))

    print(f"Built {len(orders)} orders "
          f"(crossed rows → {sum(1 for _ in range(len(crossed_df) if crossed_df is not None else 0)) * 2} orders, "
          f"remaining rows → {len(remaining_df) if remaining_df is not None else 0} orders).")
    return orders


In [ ]:
import pandas as pd
from basket_order_xml_handler import BasketOrderXMLHanlder
from base import CheckPretradeCompliance, FlowControlTag, ListProcessingLevel
import uuid

crossed_path = "crossed_trades_20251114_130344.csv"
remaining_path = "remaining_trades_20251114_130344.csv"

crossed_df = pd.read_csv(crossed_path)
remaining_df = pd.read_csv(remaining_path)

# Combine both
df_all = pd.concat([crossed_df, remaining_df], ignore_index=True, sort=False)

# Build list_of_orders using the new function
list_of_orders = build_orders_from_df(crossed_df, remaining_df)

In [ ]:
# Generate XML
xml_out = BasketOrderXMLHanlder.get_request_xml_string(
    custom_list_id=uuid.uuid1(),
    list_of_orders=list_of_orders,
    basket_name=None,
    basket_name_prefix="BQuantDemo",
    route_to_session="4571.DRAY.BQNT",
    check_pretrade_compliance=CheckPretradeCompliance.NO,
    flow_control_flag=FlowControlTag.ACTIVE_ORDER,
    list_processing_level=ListProcessingLevel.LIST,
    compliance_override_text="TestOverride",
)

# Save XML to a file
with open("basket_order_request.txt", "w", encoding="utf-8") as f:
    f.write(xml_out)


# print(xml_out)

In [ ]:
import uuid
from base import (
    Side, OrderType, SecurityIdType, SingleAllocation,
    CheckPretradeCompliance, FlowControlTag, ListProcessingLevel,
)
from basket_order_xml_handler import BasketOrderXMLHanlder

In [ ]:
list_of_orders = [
    dict(
        security_id="NVDA US Equity",
        security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
        side=Side.BUY,
        order_type=OrderType.LIMIT,
        limit_price=145.12,
        quantity=1000,
        settl_currency="USD",
        allocation_instruction=[SingleAllocation(Account="TEST", Quantity=1000)],
        crossed=True,   # <-- include broker (CROSS)
    ),
    dict(
        security_id="ABBV US Equity",
        security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
        side=Side.BUY,
        order_type=OrderType.LIMIT,
        limit_price=192.55,
        quantity=500,
        settl_currency="USD",
        allocation_instruction=[SingleAllocation(Account="TEST", Quantity=500)],
        crossed=False,  # <-- no broker / standard Parties block
    ),
]

xml_out = BasketOrderXMLHanlder.get_request_xml_string(
    custom_list_id=uuid.uuid1(),
    list_of_orders=list_of_orders,
    basket_name=None,
    basket_name_prefix="BQuantDemo",
    route_to_session="4571.DRAY.BQNT",
    check_pretrade_compliance=CheckPretradeCompliance.NO,
    flow_control_flag=FlowControlTag.ACTIVE_ORDER,
    list_processing_level=ListProcessingLevel.LIST,
    compliance_override_text="TestOverride",
)

# Save XML to a file
with open("test.txt", "w", encoding="utf-8") as f:
    f.write(xml_out)

print(xml_out)

In [ ]:
#run_request_from_file('Request-What-IF.txt', url)
#run_request_from_file('Request-Single.txt', url)
run_request_from_file('test.txt', url)
#run_request_from_file('basket_order_request.txt', url)